In [ ]:
# Создайте базу данных sqlite, можно использовать sqlalchemy или pandas, или любой другой инструмент
# Импорт необходимых библиотек
import pandas as pd
from sqlalchemy import create_engine
from  sqlalchemy.exc import OperationalError
CONN = create_engine('sqlite:///C:\\test_22.01.2023.db', echo=False)

In [5]:
from datetime import date

print(date(2018, 10, 1))

2018-10-01


In [ ]:
# Функция принимает на вход дату
# и выгружает курс валюты EUR к USD за этот день
# Функция возвращает pandas.read_csv()

from datetime import date
def extract_currency(date):
  import requests
  url = 'https://api.exchangerate.host/timeseries?start_date='+str(date)+'&end_date='+str(date)+'&base=EUR&symbols=USD&format=json'
  response = requests.get(url)
  data = response.json()
  extract_key = list(data['rates'][str(date)].keys())[0]
  extract_value = list(data['rates'][str(date)].values())[0]
  data['rates'] = str(extract_key) + ' : ' + str(extract_value)
  data['currency'] = data.pop('base')
  data['date'] = data.pop('start_date')
  result = {key: [data[key]] for key in data.keys() & {'currency', 'rates', 'date'}}
  return pd.DataFrame(result)

In [ ]:
# Функция которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция возвращает pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new

def extract_data(date):
  import requests
  url = 'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/'+str(date)+'.csv'
  res = requests.get(url)
  if str(res) == '<Response [200]>':
    data = pd.read_csv(url,index_col=0).reset_index()
    return data
  else:
    return print('No data for this date')

In [ ]:
# Функция которая принимает на вход список словарей
# и загружает в табличку в sqlite

def insert_to_db(data, table_name, conn):
  try:
    data.to_sql(table_name, con=conn)
    print('Table', table_name, 'created')
  except ValueError:
    print(table_name, 'table already exist')

In [ ]:
# Функция которая выполнит SQL запрос к базу данных

def sql_query(sql, conn):
    try:
      if sql.split()[0] != 'SELECT':
          conn.execute(sql)
          print(sql.split()[0], 'executed')
      else:
        return conn.execute(sql).fetchall()
    except OperationalError:
        print('Table does not exist')

In [ ]:
# Функция которая отправит полученный отчет на почту

def send_report(data, mail):
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    import smtplib

    HOST = "smtp.yandex.ru"
    TO = mail
    FROM = "ForStudyMailJlt@yandex.ru"
    def html_pretty(df):
        return """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
        """.format(df.to_html())
    msg = MIMEMultipart()
    part = MIMEText(html_pretty(data), 'html')
    msg.attach(part)

    server = smtplib.SMTP(HOST, 587)
    server.starttls()
    server.login("ForStudyMailJlt@yandex.ru", "Aa123123!-")
    server.sendmail("ForStudyMailJlt@yandex.ru", TO, msg.as_string())
    server.quit()
if __name__ == '__main__':
    pass
    # Send Email
    # send_report(data_st, "ForStudyMailJlt@yandex.ru")

In [ ]:
# генерацию дат
# даты с 2021-01-01 по 2021-01-04

dates_list = [str(date(2021,1,i)) for i in range(1,5)]

def main(date, email, conn):
  
  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)

  try:
    conn.execute("CREATE TABLE JOIN_DATA (currency TEXT NOT NULL,value INTEGER,date TEXT NOT NULL,rates TEXT NOT NULL)")
  except OperationalError:
    pass  

  # Вставляем данные в БД
  insert_to_db(currency, 'currency', conn)
  insert_to_db(data, 'data', conn)

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  conn.execute('INSERT INTO JOIN_DATA(currency, value, date, rates) \
                SELECT currency, value, date, rates \
                FROM (SELECT d.currency, d.value, d.date, c.rates \
                FROM currency c\
                JOIN data d USING(currency))')

  # Очистка временных таблиц CURRENCY, DATA
  sql_query('DROP TABLE currency', conn)
  sql_query('DROP TABLE data', conn)

  # Получение данных из таблицы JOIN_DATA за конкретный день
  report = pd.DataFrame(sql_query("\
                                  SELECT currency, value, date, rates \
                                  FROM JOIN_DATA WHERE date = "+"'"+str(date)+"'", conn), 
                                  columns=['currency', 'value', 'date','rates'])
  send_report(report, email)

# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date, "ForStudyMailJlt@yandex.ru", CONN)

Table currency created
Table data created
DROP executed
DROP executed
Table currency created
Table data created
DROP executed
DROP executed
Table currency created
Table data created
DROP executed
DROP executed
Table currency created
Table data created
DROP executed
DROP executed


In [ ]:
import sqlite3
g = sqlite3.connect('C:\\test_22.01.2023.db')
curs = g.cursor()
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('JOIN_DATA',)]

In [ ]:
[i for i in CONN.execute('SELECT * from JOIN_DATA').fetchall()]

[('EUR', 23, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 24, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 38, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 42, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 48, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 65, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 74, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 74, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 86, '2021-01-01', 'USD : 1.217582'),
 ('EUR', 3, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 27, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 36, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 39, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 45, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 49, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 71, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 94, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 96, '2021-01-02', 'USD : 1.2135'),
 ('EUR', 2, '2021-01-03', 'USD : 1.223409'),
 ('EUR', 23, '2021-01-03', 'USD : 1.223409'),
 ('EUR', 23, '2021-01-03', 'USD : 1.223409'),
 ('EUR', 35, '2021-01-03', 'USD : 1.223409'),
 ('EUR',